In [1]:
! nvidia-smi

Tue Dec 21 22:04:47 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:1B.0 Off |                    0 |
| N/A   44C    P0    52W / 300W |  14464MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:00:1C.0 Off |                    0 |
| N/A   

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('/app')

In [4]:
from hydra import initialize, compose
from hydra.utils import instantiate
from IPython.display import Video


with initialize(config_path="../conf"):
    cfg = compose(config_name='00_actor_critic')

In [5]:
from snake_reinforced.policy_gradient import PolicyGradientTrainer

pygame 2.1.1.dev1 (SDL 2.0.8, Python 3.7.11)
Hello from the pygame community. https://www.pygame.org/contribute.html
couldn't import doomish
Couldn't import doom


In [6]:
cfg.net._target_ = 'snake_reinforced.nn.actor_critic.ActorCriticConvNet'
cfg.logging.run_name = "pg_cnn_03_truncate_episodes"

cfg.loss.value_weight = 0.5
cfg.loss.entropy_weight = -2

cfg.device='cuda:2'

In [7]:
# cfg.training.batch_size = 8
# cfg.logging.log_video_every_n_epoch = 10

trainer = PolicyGradientTrainer(cfg, use_wandb=True)

Setting global seed to 0


ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4528:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4528:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4528:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5007:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2495:(snd_pcm_open_noupdate) Unknown PCM default
ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4528:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4528:(_snd_config_evaluate) function snd_func_concat returned error: N

In [8]:
checkpoint_path = './checkpoints/pg_cnn_01_grad_clip/snake__epoch_after_crash.pytorch'
import torch
state_dict = torch.load(checkpoint_path)

In [9]:
trainer.agent.load_state_dict(state_dict)

<All keys matched successfully>

In [10]:
cfg.training.sampling_frame_limit

32000

In [ ]:
trainer.train()

Setting global seed to 0


wandb: Currently logged in as: kilianovski (use `wandb login --relogin` to force relogin)


Started sampling episodes
Started sampling episodes


In [ ]:
from snake_reinforced.game import ACTION2ID, play_game
from snake_reinforced.vis import save_snake_video

In [ ]:
filename = './videos/t.mp4'

In [ ]:
save_snake_video(trainer.ple_env, trainer.agent, filename, n_last_frames=0)

In [ ]:
from IPython.display import Video

Video(filename)

In [ ]:
frames = play_game

In [ ]:
trainer.save_agent('after_crash')

In [ ]:
from torch.nn.functional import one_hot, log_softmax, softmax, normalize
import pandas as pd
import numpy as np
import torch
import wandb
from hydra.utils import instantiate

from snake_reinforced.game import init_ple_env, ACTION2PLE_CODE, get_snake_len
from snake_reinforced.agents import to_action_name
from snake_reinforced.vis import save_snake_video
from snake_reinforced.infrastructure import seed_all, save_state_dict
from snake_reinforced.utilities.grad import grad_norm

In [ ]:
from snake_reinforced.utilities.grad import main_params
from snake_reinforced.policy_gradient import play_episode

In [ ]:
self = trainer
optimizer = instantiate(self.cfg.optimizer, self.agent.parameters())

In [ ]:
episodes  = [play_episode(self.ple_env, self.agent, self.cfg)]

episodes = pd.DataFrame(episodes)

epoch_logits = torch.cat(
    episodes['episode_logits'].values.tolist())
epoch_weighted_log_probs = torch.cat(
    episodes['sum_weighted_log_probs'].values.tolist())
epoch_advantages = torch.cat(
    episodes['discounted_rewards_to_go'].values.tolist())

loss = self.train_epoch_step(
    epoch_logits, epoch_weighted_log_probs, epoch_advantages, 0)

In [ ]:
loss.backward()

In [ ]:

params = list(params)

In [ ]:
len(params)

In [ ]:
new_params = 

In [ ]:
new_params

In [ ]:
torch.nn.utils.clip_grad_norm_?

In [ ]:
params[0].grad

In [ ]:
p = list(self.agent.actor_critic_net.parameters())
p[0].grad

In [ ]:
self = trainer.agent.actor_critic_net
self = self.cpu()
import torch

In [ ]:
# self(x)

In [ ]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

In [ ]:
self.ln1 = nn.LayerNorm((8))

In [ ]:
x = torch.rand(1, 1, 64, 64)

x = self.conv1(x)
x = self.ln1(x)
x = F.relu(x)
x = F.max_pool2d(x, 2)

print(x.shape)

x = self.conv2(x)
print(x.shape)

x = F.relu(x)

x = F.max_pool2d(x, 2)

print(x.shape)
x = self.dropout1(x)
x = torch.flatten(x, 1)
print(x.shape)


In [ ]:
from torch import nn
nn.Conv2d?

In [ ]:
123